In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
from pymongo import MongoClient
from bson.objectid import ObjectId

import time,datetime

import pandas as pd
import pandas_datareader.data as web
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import style